In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
from shiny import App, render, ui

In [2]:
# Load and process data
df = pd.read_csv("2022_Q1_OR_Utilization.csv")
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172 entries, 0 to 2171
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              2172 non-null   int64 
 1   Encounter ID       2172 non-null   int64 
 2   Date               2172 non-null   object
 3   OR Suite           2172 non-null   int64 
 4   Service            2172 non-null   object
 5   CPT Code           2172 non-null   int64 
 6   CPT Description    2172 non-null   object
 7   Booked Time (min)  2172 non-null   int64 
 8   OR Schedule        2172 non-null   object
 9   Wheels In          2172 non-null   object
 10  Start Time         2172 non-null   object
 11  End Time           2172 non-null   object
 12  Wheels Out         2172 non-null   object
dtypes: int64(5), object(8)
memory usage: 220.7+ KB


(   index  Encounter ID    Date  OR Suite      Service  CPT Code  \
 0      0         10001  1/3/22         1     Podiatry     28110   
 1      1         10002  1/3/22         1     Podiatry     28055   
 2      2         10003  1/3/22         1     Podiatry     28297   
 3      3         10004  1/3/22         1     Podiatry     28296   
 4      4         10005  1/3/22         2  Orthopedics     27445   
 
                              CPT Description  Booked Time (min)   OR Schedule  \
 0   Partial ostectomy, fifth metatarsal head                 90   1/3/22 7:00   
 1  Neurectomy, intrinsic musculature of foot                 60   1/3/22 8:45   
 2                       Lapidus bunionectomy                150  1/3/22 10:00   
 3         Bunionectomy with distal osteotomy                120  1/3/22 12:45   
 4        Arthroplasty, knee, hinge prothesis                120   1/3/22 7:00   
 
       Wheels In    Start Time      End Time    Wheels Out  
 0   1/3/22 7:05   1/3/22 7:32   1/

In [3]:
# convert column names to lowercase and replace space and parenthese with underscore
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace(r'[()]', '', regex=True)

#  display updated column names
print(df.columns)


Index(['index', 'encounter_id', 'date', 'or_suite', 'service', 'cpt_code',
       'cpt_description', 'booked_time_min', 'or_schedule', 'wheels_in',
       'start_time', 'end_time', 'wheels_out'],
      dtype='object')


In [4]:
# convert 'date' column to datetime objects
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
#df['start_time'] = pd.to_datetime(df['start_time'], format='%m/%d/%y %H:%M') 

columns_to_convert = ['start_time', 'end_time', 'wheels_in', 'wheels_out', 'or_schedule']

for col in columns_to_convert:
    try:
        df[col] = pd.to_datetime(df[col], format='%m/%d/%y %H:%M')
    except ValueError:
        print(f"Warning: Unable to convert {col} to datetime.")
        pass

# Display the updated column names to verify
print(df.columns)


    
# find latest date in the dataset
last_day = df['date'].max()

first_day = df['date'].min()

print(f"Latest date in the dataset: {last_day}")
print(f"First date in the dataset: {first_day}")

Index(['index', 'encounter_id', 'date', 'or_suite', 'service', 'cpt_code',
       'cpt_description', 'booked_time_min', 'or_schedule', 'wheels_in',
       'start_time', 'end_time', 'wheels_out'],
      dtype='object')
Latest date in the dataset: 2022-03-31 00:00:00
First date in the dataset: 2022-01-03 00:00:00


In [5]:
# # filter df to only include records from last day
# last_day_data = df[df['date'] == last_day]

# last_day_data.head()

In [6]:
# unique OR suites on last day
unique_or = df[df['date'] == last_day]['or_suite'].unique()
unique_or

array([1, 2, 3, 4, 5, 6, 7, 8])

In [7]:
# identify start of day on last day
start_of_day = df[df['date'] == last_day]['or_schedule'].min()
start_of_day

Timestamp('2022-03-31 07:00:00')

In [8]:
# or status at the beginning of the last day

or_status = {}

for suite in unique_or:
    suite_data = df[df['or_suite'] == suite]
    if not suite_data.empty and (suite_data['start_time'] >= start_of_day).any():
        or_status[suite] = "In Use/Scheduled"
    else:
        or_status[suite] = "Available"
    
# turn or_status into a dataframe
or_status_df = pd.DataFrame(list(or_status.items()), columns=['OR Suite', 'Status'])
or_status_df



,OR Suite,Status
0,1,In Use/Scheduled
1,2,In Use/Scheduled
2,3,In Use/Scheduled
3,4,In Use/Scheduled
4,5,In Use/Scheduled
5,6,In Use/Scheduled
6,7,In Use/Scheduled
7,8,In Use/Scheduled


In [9]:
# extract time from or_schedule, start_time, end_time, wheels_in, wheels_out
df['or_schedule_time'] = pd.to_datetime(df['or_schedule']).dt.time
df['start_time_time'] = pd.to_datetime(df['start_time']).dt.time
df['end_time_time'] = pd.to_datetime(df['end_time']).dt.time
df['wheels_in_time'] = pd.to_datetime(df['wheels_in']).dt.time
df['wheels_out_time'] = pd.to_datetime(df['wheels_out']).dt.time

# extract month name
df['month'] = df['date'].dt.month_name()

#duration of cases
df['duration'] = df['end_time'] - df['start_time']

df.head()

,index,encounter_id,date,or_suite,service,cpt_code,cpt_description,booked_time_min,or_schedule,wheels_in,start_time,end_time,wheels_out,or_schedule_time,start_time_time,end_time_time,wheels_in_time,wheels_out_time,month,duration
0,0,10001,2022-01-03,1,Podiatry,28110,"Partial ostectomy, fifth metatarsal head",90,2022-01-03 07:00:00,2022-01-03 07:05:00,2022-01-03 07:32:00,2022-01-03 09:05:00,2022-01-03 09:17:00,07:00:00,07:32:00,09:05:00,07:05:00,09:17:00,January,0 days 01:33:00
1,1,10002,2022-01-03,1,Podiatry,28055,"Neurectomy, intrinsic musculature of foot",60,2022-01-03 08:45:00,2022-01-03 09:48:00,2022-01-03 10:13:00,2022-01-03 11:01:00,2022-01-03 11:12:00,08:45:00,10:13:00,11:01:00,09:48:00,11:12:00,January,0 days 00:48:00
2,2,10003,2022-01-03,1,Podiatry,28297,Lapidus bunionectomy,150,2022-01-03 10:00:00,2022-01-03 11:50:00,2022-01-03 12:20:00,2022-01-03 12:42:00,2022-01-03 12:58:00,10:00:00,12:20:00,12:42:00,11:50:00,12:58:00,January,0 days 00:22:00
3,3,10004,2022-01-03,1,Podiatry,28296,Bunionectomy with distal osteotomy,120,2022-01-03 12:45:00,2022-01-03 13:29:00,2022-01-03 13:53:00,2022-01-03 14:50:00,2022-01-03 15:02:00,12:45:00,13:53:00,14:50:00,13:29:00,15:02:00,January,0 days 00:57:00
4,4,10005,2022-01-03,2,Orthopedics,27445,"Arthroplasty, knee, hinge prothesis",120,2022-01-03 07:00:00,2022-01-03 07:15:00,2022-01-03 07:50:00,2022-01-03 09:38:00,2022-01-03 09:51:00,07:00:00,07:50:00,09:38:00,07:15:00,09:51:00,January,0 days 01:48:00


In [10]:
# which case has the shortest duration
shortest_case = df.loc[df['duration'].idxmin()]
shortest_case

# which case has the longest duration
longest_case = df.loc[df['duration'].idxmax()]
longest_case

# average duration of cases
average_duration = df['duration'].mean()
average_duration





Timedelta('0 days 00:45:28.508287292')

In [11]:
# count monthly case volume by group by service, or_suite and month
monthly_case_volume = df.groupby(['service', 'or_suite', 'month'])['encounter_id'].nunique().reset_index()

# # rename count column
monthly_case_volume.rename(columns={'encounter_id': 'case_volume'}, inplace=True)

# # display result
print(monthly_case_volume)

          service  or_suite     month  case_volume
0             ENT         5  February           62
1             ENT         5   January           62
2             ENT         5     March           73
3         General         8  February           33
4         General         8   January           42
5         General         8     March           42
6           OBGYN         4  February           52
7           OBGYN         4   January           48
8           OBGYN         4     March           64
9   Ophthalmology         3  February          107
10  Ophthalmology         3   January           95
11  Ophthalmology         3     March          132
12    Orthopedics         2  February           79
13    Orthopedics         2   January           79
14    Orthopedics         2     March           94
15    Orthopedics         8  February           24
16    Orthopedics         8   January           18
17    Orthopedics         8     March           27
18     Pediatrics         3  Fe

In [12]:
# count cpt_description by month
monthly_cpt_description = df.groupby(['cpt_description', 'month'])['encounter_id'].nunique().reset_index()

# # rename count column
monthly_cpt_description.rename(columns={'encounter_id': 'case_volume'}, inplace=True)

# count total cases by cpt_code
total_cases_by_cpt_code = df.groupby(['cpt_code'])['encounter_id'].nunique().reset_index()

# # rename count column
total_cases_by_cpt_code.rename(columns={'encounter_id': 'case_volume'}, inplace=True)

# # display result
print(total_cases_by_cpt_code)

# extract most common cpt_code cpt_description
most_common_cpt_code = df['cpt_code'].mode()[0]
most_common_cpt_description = df[df['cpt_code'] == most_common_cpt_code]['cpt_description'].mode()[0]


df.head()

    cpt_code  case_volume
0      14060           86
1      15773           36
2      17110           69
3      26045           21
4      26356           20
5      26735           21
6      27130           23
7      27445           82
8      28055           18
9      28060           42
10     28110           18
11     28285           42
12     28289           23
13     28296           85
14     28297           18
15     28820           78
16     29877          112
17     30400           16
18     30520           46
19     36901           95
20     42826          151
21     43775           78
22     47562           39
23     52353           76
24     55250           78
25     55873           39
26     57460           82
27     58562           82
28     64721           42
29     66982          334
30     69421           88
31     69436          132


,index,encounter_id,date,or_suite,service,cpt_code,cpt_description,booked_time_min,or_schedule,wheels_in,start_time,end_time,wheels_out,or_schedule_time,start_time_time,end_time_time,wheels_in_time,wheels_out_time,month,duration
0,0,10001,2022-01-03,1,Podiatry,28110,"Partial ostectomy, fifth metatarsal head",90,2022-01-03 07:00:00,2022-01-03 07:05:00,2022-01-03 07:32:00,2022-01-03 09:05:00,2022-01-03 09:17:00,07:00:00,07:32:00,09:05:00,07:05:00,09:17:00,January,0 days 01:33:00
1,1,10002,2022-01-03,1,Podiatry,28055,"Neurectomy, intrinsic musculature of foot",60,2022-01-03 08:45:00,2022-01-03 09:48:00,2022-01-03 10:13:00,2022-01-03 11:01:00,2022-01-03 11:12:00,08:45:00,10:13:00,11:01:00,09:48:00,11:12:00,January,0 days 00:48:00
2,2,10003,2022-01-03,1,Podiatry,28297,Lapidus bunionectomy,150,2022-01-03 10:00:00,2022-01-03 11:50:00,2022-01-03 12:20:00,2022-01-03 12:42:00,2022-01-03 12:58:00,10:00:00,12:20:00,12:42:00,11:50:00,12:58:00,January,0 days 00:22:00
3,3,10004,2022-01-03,1,Podiatry,28296,Bunionectomy with distal osteotomy,120,2022-01-03 12:45:00,2022-01-03 13:29:00,2022-01-03 13:53:00,2022-01-03 14:50:00,2022-01-03 15:02:00,12:45:00,13:53:00,14:50:00,13:29:00,15:02:00,January,0 days 00:57:00
4,4,10005,2022-01-03,2,Orthopedics,27445,"Arthroplasty, knee, hinge prothesis",120,2022-01-03 07:00:00,2022-01-03 07:15:00,2022-01-03 07:50:00,2022-01-03 09:38:00,2022-01-03 09:51:00,07:00:00,07:50:00,09:38:00,07:15:00,09:51:00,January,0 days 01:48:00


In [20]:
# Calculate turnover time in minutes

df["turnover_time"] = df.groupby(["date", "or_suite"]).apply(
    lambda group: group["wheels_in"] - group["wheels_out"].shift(1)).reset_index(level=[0, 1], drop=True)

df.head(10)

/var/folders/_v/nfpvjvdn7670tj6vbvh73bjr0000gn/T/ipykernel_1651/3497960528.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["turnover_time"] = df.groupby(["date", "or_suite"]).apply(


,index,encounter_id,date,or_suite,service,cpt_code,cpt_description,booked_time_min,or_schedule,wheels_in,...,end_time,wheels_out,or_schedule_time,start_time_time,end_time_time,wheels_in_time,wheels_out_time,month,duration,turnover_time
0,0,10001,2022-01-03,1,Podiatry,28110,"Partial ostectomy, fifth metatarsal head",90,2022-01-03 07:00:00,2022-01-03 07:05:00,...,2022-01-03 09:05:00,2022-01-03 09:17:00,07:00:00,07:32:00,09:05:00,07:05:00,09:17:00,January,0 days 01:33:00,NaT
1,1,10002,2022-01-03,1,Podiatry,28055,"Neurectomy, intrinsic musculature of foot",60,2022-01-03 08:45:00,2022-01-03 09:48:00,...,2022-01-03 11:01:00,2022-01-03 11:12:00,08:45:00,10:13:00,11:01:00,09:48:00,11:12:00,January,0 days 00:48:00,0 days 00:31:00
2,2,10003,2022-01-03,1,Podiatry,28297,Lapidus bunionectomy,150,2022-01-03 10:00:00,2022-01-03 11:50:00,...,2022-01-03 12:42:00,2022-01-03 12:58:00,10:00:00,12:20:00,12:42:00,11:50:00,12:58:00,January,0 days 00:22:00,0 days 00:38:00
3,3,10004,2022-01-03,1,Podiatry,28296,Bunionectomy with distal osteotomy,120,2022-01-03 12:45:00,2022-01-03 13:29:00,...,2022-01-03 14:50:00,2022-01-03 15:02:00,12:45:00,13:53:00,14:50:00,13:29:00,15:02:00,January,0 days 00:57:00,0 days 00:31:00
4,4,10005,2022-01-03,2,Orthopedics,27445,"Arthroplasty, knee, hinge prothesis",120,2022-01-03 07:00:00,2022-01-03 07:15:00,...,2022-01-03 09:38:00,2022-01-03 09:51:00,07:00:00,07:50:00,09:38:00,07:15:00,09:51:00,January,0 days 01:48:00,NaT
5,5,10006,2022-01-03,2,Orthopedics,27445,"Arthroplasty, knee, hinge prothesis",120,2022-01-03 09:15:00,2022-01-03 10:25:00,...,2022-01-03 12:34:00,2022-01-03 12:52:00,09:15:00,11:00:00,12:34:00,10:25:00,12:52:00,January,0 days 01:34:00,0 days 00:34:00
6,6,10007,2022-01-03,3,Ophthalmology,66982,Extracapsular cataract removal,45,2022-01-03 07:00:00,2022-01-03 07:03:00,...,2022-01-03 07:31:00,2022-01-03 07:38:00,07:00:00,07:13:00,07:31:00,07:03:00,07:38:00,January,0 days 00:18:00,NaT
7,7,10008,2022-01-03,3,Ophthalmology,66982,Extracapsular cataract removal,45,2022-01-03 08:00:00,2022-01-03 08:00:00,...,2022-01-03 08:30:00,2022-01-03 08:41:00,08:00:00,08:12:00,08:30:00,08:00:00,08:41:00,January,0 days 00:18:00,0 days 00:22:00
8,8,10009,2022-01-03,3,Ophthalmology,66982,Extracapsular cataract removal,45,2022-01-03 09:00:00,2022-01-03 09:04:00,...,2022-01-03 09:30:00,2022-01-03 09:39:00,09:00:00,09:15:00,09:30:00,09:04:00,09:39:00,January,0 days 00:15:00,0 days 00:23:00
9,9,10010,2022-01-03,3,Ophthalmology,66982,Extracapsular cataract removal,45,2022-01-03 10:00:00,2022-01-03 10:01:00,...,2022-01-03 10:26:00,2022-01-03 10:34:00,10:00:00,10:12:00,10:26:00,10:01:00,10:34:00,January,0 days 00:14:00,0 days 00:22:00


In [22]:
# average overall turnover time inper OR suite 
average_turnover_time = df['turnover_time'].mean()
average_turnover_time

Timedelta('0 days 00:29:52.374701670')

In [15]:
# save transfromed df to csv
df.to_csv('df_transformed.csv')